In [1]:
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np


In [2]:
def create_folder(path):
    try:
        os.mkdir(path)
    except:
        pass

In [3]:
font_color = '#525252'
hfont = {'fontname':'Calibri'}

facecolor = '#eaeaf2'
color_red = '#fd625e'
color_blue = '#01b8aa'

def plot_graphs(df,country,save_fig_path,API_NAME):
    try:
        os.mkdir(save_fig_path)
    except:
        pass    
    index = ['ORIGINAL','GREYSCALE','RGB-0.3','RGB-0.5','SPREAD','MASKED']
    
    df_res = df.copy()
    df_res = df_res[[0]+index]
    df_res.to_csv(save_fig_path.split('gender_wise')[0]+'accuracy_'+country+'.csv',index = False)
    
    column0 = []
    column1 = []
    for col in index:
        column0.append(df[col][1])
        column1.append(df[col][2])
    title0 = 'Male'
    title1 = 'Female'
    
    fig, axes = plt.subplots(figsize=(10,5), facecolor=facecolor, ncols=2, sharey=True)
    fig.tight_layout()

    axes[0].barh(index, column0, align='center', color=color_red)
    axes[0].set_title(title0, fontsize=18, pad=15, color=color_red, **hfont)
    axes[1].barh(index, column1, align='center', color=color_blue)
    axes[1].set_title(title1, fontsize=18, pad=15, color=color_blue, **hfont)
    
    # If you have positive numbers and want to invert the x-axis of the left plot
    axes[0].invert_xaxis() 

    # To show data from highest to lowest
    plt.gca().invert_yaxis()


    axes[0].set(yticks=index, yticklabels=index)
    axes[0].yaxis.tick_left()
    axes[0].tick_params(axis='y', colors='white') # tick color

    ticks = list(np.linspace(0,100,11,dtype = 'int'))
    axes[1].set_xticks(ticks)
    axes[1].set_xticklabels(ticks)

    axes[0].set_xticks(ticks)
    axes[0].set_xticklabels(ticks)

    for label in (axes[0].get_xticklabels() + axes[0].get_yticklabels()):
        label.set(fontsize=13, color=font_color, **hfont)
    for label in (axes[1].get_xticklabels() + axes[1].get_yticklabels()):
        label.set(fontsize=13, color=font_color, **hfont)
        

    if API_NAME == 'deepface' or API_NAME == 'deepface-fine':
        for i, acc in enumerate(column0):
            axes[0].text(acc+15,i-0.05, acc, color='black', fontweight='bold',fontsize = 12)

        for i, acc in enumerate(column1):
            axes[1].text(acc+2,i-0.05, acc, color='black', fontweight='bold',fontsize = 12)
    else:
        for i, acc in enumerate(column0):
            axes[0].text(acc-2,i-0.1, acc, color='black', fontweight='bold',fontsize = 12)

        for i, acc in enumerate(column1):
            axes[1].text(acc-15,i-0.1, acc, color='black', fontweight='bold',fontsize = 12)

    plt.subplots_adjust(wspace=0, top=0.85, bottom=0.1, left=0.18, right=0.95)
    
    fig.supxlabel("Accuracy (%)")
    fig.supylabel("Image Type")
    fig.suptitle(country, fontsize=15)
    plt.savefig(save_fig_path+country+'.png',bbox_inches ="tight",dpi = 150)
    plt.close(fig)
#     plt.show()

In [4]:
def plot_graph_overall(df,country,save_fig_path,API_NAME):
    try:
        os.mkdir(save_fig_path)
    except:
        pass
    
    index = ['ORIGINAL','GREYSCALE','RGB-0.3','RGB-0.5','SPREAD','MASKED']
    values = []
    for col in index:
        values.append(df[col][0])
    fig, ax = plt.subplots(figsize = (8, 4),facecolor=facecolor)
    ax.bar(index,values,color = "#0A4D68")
    
    if API_NAME == 'deepface' or API_NAME == 'deepface-fine':
        for i,acc in enumerate(values):
            ax.text(i-0.2,acc+5,acc,color='black', fontweight='bold',fontsize = 12)
    else:
        for i,acc in enumerate(values):
            ax.text(i-0.2,acc-6.5,acc,color='white', fontweight='bold',fontsize = 12)
    
    plt.ylim(0,100)
    plt.xlabel("Image Type")
    plt.ylabel("Accuracy (%)")
    plt.title(country)
    plt.savefig(save_fig_path+country+"_overall"+'.jpg',dpi = 150)
    plt.close()
#     plt.show()

In [14]:
def doAnalysis(country,res_csv,test_data_path,API_NAME,save_fig_path):
    df_res = pd.read_csv(res_csv)
    for i in range(len(df_res)):
        df_res.at[i,'name'] = df_res['name'][i]+'_'+country
    test_data = os.listdir(test_data_path)
    for i in range(len(test_data)):
        test_data[i] = test_data[i].split('.')[0]
#     print(test_data)
    ## Keep only those result which are in the test dataset
    df_copy = df_res[df_res['name'].isin(test_data)]
#     print(len(df_copy))
#     print(df_copy)
#     return

    df_res = pd.DataFrame(['Overall','Male','Female'])
    columns = df_copy.columns[2:]
    for column in columns:
        df_res[column.title()] = None
        total = len(df_copy)
        df_male = df_copy.copy()
        df_male = df_male[df_male['gender'] == 'M']
        df_female = df_copy.copy()
        df_female = df_female[df_female['gender'] == 'F']

        total_male = len(df_male)
        total_female = len(df_female)

        total_correct_prediction = len(df_copy[df_copy['gender'] == df_copy[column]])
        correct_prediction_male = len(df_male[df_male['gender'] == df_male[column]])
        correct_prediction_female = len(df_female[df_female['gender'] == df_female[column]])

        overall_accuracy = (total_correct_prediction/total)*100
        male_accuracy = (correct_prediction_male/total_male)*100
        female_accuracy = (correct_prediction_female/total_female)*100

        df_res[column.title()][0] = round(overall_accuracy,2)
        df_res[column.title()][1] = round(male_accuracy,2)
        df_res[column.title()][2] = round(female_accuracy,2)
    if API_NAME == 'aws':
        df_res=df_res.rename(columns = {'Normal':'ORIGINAL','Masked':'MASKED','Greyish':'GREYSCALE',
                           'Rgb0.3':'RGB-0.3','Rgb0.5':'RGB-0.5','Spread':'SPREAD'})
    elif API_NAME == 'facepp':
        df_res=df_res.rename(columns = {'Original Normal':'ORIGINAL','Original Masked':'MASKED','Greyish Normal':'GREYSCALE',
                           'Rgb0.3 Normal':'RGB-0.3','Rgb0.5 Normal':'RGB-0.5','Spread Normal':'SPREAD'})
    elif API_NAME == 'azure':
        df_res=df_res.rename(columns = {'Normal':'ORIGINAL','Masked':'MASKED','Greyish':'GREYSCALE',
                           'Rgb0.3':'RGB-0.3','Rgb0.5':'RGB-0.5','Spread':'SPREAD'})
    elif API_NAME == 'deepface' or API_NAME == 'deepface-fine':
        df_res=df_res.rename(columns = {'Normal':'ORIGINAL','Masked':'MASKED','Greyish':'GREYSCALE',
                           'Rgb0.3 Noise':'RGB-0.3','Rgb0.5 Noise':'RGB-0.5','Spread Noise':'SPREAD'})
    else: ## libfaceid
        df_res=df_res.rename(columns = {'Normal':'ORIGINAL','Masked':'MASKED','Greyish':'GREYSCALE',
                           'Rgb0.3':'RGB-0.3','Rgb0.5':'RGB-0.5','Spread':'SPREAD'})
#     print(df_res)
    df_res.to_csv(save_fig_path+'accuracy_'+country+'.csv',index = False)
    try:
        create_folder(save_fig_path+"gender_wise")
        plot_graphs(df_res,country,save_fig_path+"gender_wise/",API_NAME)
        create_folder(save_fig_path+"overall")
        plot_graph_overall(df_res,country,save_fig_path+"overall/",API_NAME)
    except:
        print("Error in Plotting")
    return df_copy

In [15]:
def overall_accuracy(df,save_fig_path):
    img_types = df.columns[2:]
#     print(df.columns)
#     print(img_types)
    df_copy = df.copy()
    df_res = pd.DataFrame()
    df_res['image type'] = ['male','female','combined']
    for img_type in img_types:
        print("Image Type : "+img_type)
        print(len(df_copy))
        total = len(df_copy)
        df_male = df_copy[df_copy['gender'] == 'M']
        df_female = df_copy[df_copy['gender'] == 'F']

        total_male = len(df_male)
        total_female = len(df_female)

        predicted_male = len(df_male[df_male[img_type] == 'M'])
        predicted_female = len(df_female[df_female[img_type] == 'F'])

        total_correct_prediction = len(df_copy[df_copy['gender'] == df_copy[img_type]])
#         print("Total : ",total)
#         print("Total correct prediction : ",total_correct_prediction)
        
        
        
        print("Overall Male accuracy : %0.2f"%((predicted_male/total_male)*100))
        print("Overall Female accuracy : %0.2f"%((predicted_female/total_female)*100))
        print("Overall Accuracy : %0.2f"%((total_correct_prediction/total)*100))
        print()
        lis = ["%0.2f"%((predicted_male/total_male)*100),"%0.2f"%((predicted_female/total_female)*100),"%0.2f"%((total_correct_prediction/total)*100)]
        df_res[img_type] = lis
    print(df_res)
    df_res.to_csv(save_fig_path+"overall_accuracy_all.csv",index = False)

In [16]:
run = 1
num_img_category = 1

img_type = "original" ## trained images type

res_csv_path = "./../Test Data/result_in_csv_file/"+img_type
save_fig_path = "./../Test Data/plots_v2/"
test_data_path = 'test_data_trained_on_'+img_type

for run in [1,2,3]:
    for num_img_category in [1,2]:
        print(run,num_img_category)
        res_csv_path_new = res_csv_path+'/'+'run_'+str(run)+'/'+str(num_img_category)+'_shot/'
        save_fig_path_new = save_fig_path+'/'+img_type
        create_folder(save_fig_path_new)
        save_fig_path_new = save_fig_path_new+'/'+'run_'+str(run)
        create_folder(save_fig_path_new)
        save_fig_path_new = save_fig_path_new+'/'+str(num_img_category)+'_shot'
        create_folder(save_fig_path_new)
        countries = os.listdir(res_csv_path_new)
        print(res_csv_path_new)
        df_overall = pd.DataFrame()
        for country in countries:
            country = country.split('.')[0]
            print(country)
            df = doAnalysis(country,res_csv_path_new+'/'+country+'.csv',test_data_path,"deepface-fine",save_fig_path_new+'/')
            df_overall = df_overall.append(df)

        df_overall = df_overall.reset_index()
        df_overall = df_overall.drop('index',axis = 1)
#         overall_accuracy(df_overall,save_fig_path_new+'/')


1 1
./../Test Data/result_in_csv_file/original/run_1/1_shot/
Australia
Bangladesh
England
India
New Zealand
Pakistan
South Africa
West Indies
1 2
./../Test Data/result_in_csv_file/original/run_1/2_shot/
Australia
Bangladesh
England
India
New Zealand
Pakistan
South Africa
West Indies
2 1
./../Test Data/result_in_csv_file/original/run_2/1_shot/
Australia
Bangladesh
England
India
New Zealand
Pakistan
South Africa
West Indies
2 2
./../Test Data/result_in_csv_file/original/run_2/2_shot/
Australia
Bangladesh
England
India
New Zealand
Pakistan
South Africa
West Indies
3 1
./../Test Data/result_in_csv_file/original/run_3/1_shot/
Australia
Bangladesh
England
India
New Zealand
Pakistan
South Africa
West Indies
3 2
./../Test Data/result_in_csv_file/original/run_3/2_shot/
Australia
Bangladesh
England
India
New Zealand
Pakistan
South Africa
West Indies


## Just for simply copying the result into google sheet

In [8]:

def save_all_accuracy(save_fig_path,files):
    df_all = pd.DataFrame()
    df_all['country'] = None
    for file in files: # countries
        pref = file.split("_")[0]
        if pref != 'accuracy':
            continue
#         print(save_fig_path+file)
        df = pd.read_csv(save_fig_path+'/'+file)
    #     print(df)
    #     break
        country = file.split("_")[-1].split(".")[0]
    #     print(country)
        img_types = df.columns[1:]
        if len(df_all.columns) == 1:
            all_columns = []
            for img_type in img_types:
                all_columns.append('male-'+img_type)
                all_columns.append('female-'+img_type)
                all_columns.append('overall-'+img_type)
        #     print(all_columns)
            for col in all_columns:
                df_all[col] = None
        data = [country]
        for img_type in img_types:
            data.append(df[img_type][1])
            data.append(df[img_type][2])
            data.append(df[img_type][0])
        print(data)
        df_all.loc[len(df_all.index)] = data

    df_all.to_csv(save_fig_path+"all_accuracy.csv",index = False)
    


In [9]:
for run in [1,2,3]:
    for num_img_category in [1,2]:
        save_fig_path_new = save_fig_path+img_type
        save_fig_path_new = save_fig_path_new+'/'+'run_'+str(run)
        save_fig_path_new = save_fig_path_new+'/'+str(num_img_category)+'_shot/'
#         print(save_fig_path_new)
        files = os.listdir(save_fig_path_new)
        save_all_accuracy(save_fig_path_new,files)

['Australia', 57.5, 45.0, 53.33, 62.5, 45.0, 56.67, 35.0, 5.0, 25.0, 7.5, 0.0, 5.0, 45.0, 15.0, 35.0, 12.5, 15.0, 13.33]
['Bangladesh', 47.5, 35.0, 43.33, 55.0, 45.0, 51.67, 22.5, 15.0, 20.0, 5.0, 0.0, 3.33, 55.0, 15.0, 41.67, 15.0, 15.0, 15.0]
['England', 40.0, 60.0, 46.67, 50.0, 65.0, 55.0, 15.0, 10.0, 13.33, 5.0, 0.0, 3.33, 37.5, 30.0, 35.0, 2.5, 15.0, 6.67]
['India', 50.0, 50.0, 50.0, 50.0, 45.0, 48.33, 25.0, 5.0, 18.33, 5.0, 0.0, 3.33, 50.0, 10.0, 36.67, 17.5, 30.0, 21.67]
['New Zealand', 50.0, 25.0, 41.67, 47.5, 30.0, 41.67, 25.0, 5.0, 18.33, 2.5, 0.0, 1.67, 47.5, 20.0, 38.33, 15.0, 0.0, 10.0]
['Pakistan', 42.5, 40.0, 41.67, 45.0, 45.0, 45.0, 22.5, 0.0, 15.0, 12.5, 0.0, 8.33, 50.0, 10.0, 36.67, 20.0, 35.0, 25.0]
['South Africa', 70.0, 60.0, 66.67, 70.0, 55.0, 65.0, 35.0, 15.0, 28.33, 7.5, 0.0, 5.0, 60.0, 5.0, 41.67, 15.0, 15.0, 15.0]
['West Indies', 45.0, 40.0, 43.33, 45.0, 55.0, 48.33, 7.5, 0.0, 5.0, 0.0, 0.0, 0.0, 37.5, 10.0, 28.33, 7.5, 5.0, 6.67]
['Australia', 57.5, 50.0, 55.

## Get the average of all 1-Shot, and 2-shot for different run